In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import pandas as pd
import xgboost as xgb
from fastai.vision.all import *
from ENSEMBLE_SCORING_ALGORITHM import *

C:\Users\mindf\anaconda3\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
#Initialize all Gradient Boosted Trees
intialize_trees()

In [3]:
#Import Generator
from GENERATOR_HEALTHCARE_MODEL import LeverGenerator

#Generator model
generator_model = LeverGenerator(1200)
#Configure loss function
loss_func = nn.MSELoss()
#Configure Optimizer
optimizer = torch.optim.Adam(generator_model.parameters(), lr = 0.01)

#Training script for PyTorch (training and validation loops)
def train_model(epochs, generator):
    train_accuracies = []
    train_losses = []
    valid_accuracies = []
    valid_losses = []
    #Training Loop
    iteration_loop(training = True, epochs = epochs, generator = generator, losses = train_losses, accuracies = train_accuracies)
    #Validation Loop
    iteration_loop(training = False, epochs = epochs, generator = generator, losses = valid_losses, accuracies = valid_accuracies)
    return train_accuracies, train_losses

def iteration_loop(training, epochs, generator, losses, accuracies):
    epoch_losses = []
    total_loss = 0.0
    for epoch in range(epochs): 
        #Set gradients to zero
        generator.zero_grad()
        inputs = torch.rand((1, 1200))
        output = generator(inputs)
        #Air quality metrics
        aq_metrics = output[:13]
        exercise_metrics = output[14:16]
        #Blood report metrics
        blood_report_metrics = output[17:31]
        #Nutrition metrics
        nutrition_metrics = output[32:37]
        #Sleep metrics
        sleep_metrics = output[37:40]
        METRICS = {'pollution_gradient_tree' : aq_metrics, 'exercise': exercise_metrics,
                   'nutrition_gradient_tree': nutrition_metrics, 'sleep_gradient_tree': sleep_metrics, 
                   'blood_gradient_tree': blood_report_metrics}
        avg_score = get_avg_model_score(METRICS)
        loss = 1 - avg_score
        if not training:
            loss.backward()
        total_loss += loss
        if epoch == epochs - 1:
            avg_loss = total_loss / epochs
            print("TRAINING [{}] LOSS: ", avg_loss)    

In [4]:
#Train the model
train_model(epochs = 10, generator = generator_model)

tensor([[-0.0118, -0.0528,  0.0469, -0.0351,  0.0411, -0.0126, -0.0037,  0.0009,
         -0.0076,  0.0175, -0.0760,  0.0265, -0.0156, -0.0297, -0.0277,  0.0353,
         -0.0104,  0.0131, -0.0030, -0.0387, -0.0597, -0.0299,  0.0071,  0.0607,
         -0.0353,  0.0024, -0.0472,  0.0271,  0.0427,  0.0261, -0.0349, -0.0558,
         -0.0338,  0.0505,  0.0317, -0.0137,  0.0288, -0.0307,  0.0297, -0.0469]],
       grad_fn=<SliceBackward>)
DATASETS\POLLUTION\GEOGRAPHY_AIR_QUALITY_DATASET.pkl
DATA:  <bound method NDFrame.head of       PM2.5   PM10     NO    NO2    Nox    NH3    CO    SO2     O3  Benzene  \
0     27.93  95.68   9.72   4.71  14.19   0.88  0.79   9.36  30.18     3.39   
1     34.05  90.68  16.53   9.83  25.86   1.68  0.93   4.27  17.23     5.27   
2     38.10  61.67   7.61   3.30  10.90   1.37  0.65  14.74  26.14     7.14   
3     40.41  67.91  10.52   3.96  12.79   3.76  0.55   9.97  28.60     4.46   
4     27.21  62.04   8.60   7.20  10.64   5.90  0.53   9.63  34.72     3.57 

XGBoostError: [22:39:58] C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1258: Check failed: learner_model_param_.num_feature >= p_fmat->Info().num_col_ (13 vs. 40) : Number of columns does not match number of features in booster.